# 综述

矩阵分解+变换，以线性时间解决稀疏数据下训练特征交叉参数

# 优化目标(2阶为例)

$$
\hat{y}(\mathbf{x}) = w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j= n + 1}^{n}w_{ij}x_ix_j\\
=w_0 + \sum_{i=1}^{n}w_ix_i + \sum_{i=1}^{n}\sum_{j= n + 1}^{n}<v_i, v_j>x_ix_j
$$

实对称矩阵-正定矩阵$W$,只要k足够大，存在$W=V \cdot V^T$。如果选择足够大的k，则FM可以表示任何 interaction matrix W（交叉特征参数矩阵）。 但是，在稀疏环境中，通常应该选择一个小的k($k << n$，需要自己确定)，因为没有足够的数据来估计复杂的交互作用W。限制k会导致更好的泛化，从而改善稀疏性下的$W$。

$$
W = \begin{bmatrix}
w_{11} & w_{12} & \cdots & w_{1n}\\ 
w_{21} & w_{22} & \cdots & w_{2n}\\ 
\cdots & \cdots & \cdots & \cdots\\ 
w_{n1} & w_{n2} & \cdots & w_{nn}
\end{bmatrix} = V \cdot V^T
$$
其中，$v=[V_1, V_2, ..., V_n]$，$V_i$为k维向量。参数数量显著减少。降为$O(kn^2)$,还可以通过调整:
$$
\sum_{i=1}^{n}\sum_{j= n + 1}^{n}<v_i, v_j>x_ix_j = \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2 - (\sum_{i=1}^{n}v_{i,f}^2 x_i^2)^2)
$$
于是时间复杂度变为$O(kn)$：
$$
\hat{y}(\mathbf{x}) = w_0 + \sum_{i=1}^{n}w_ix_i + \frac{1}{2}\sum_{f=1}^{k}((\sum_{i=1}^{n}v_{i,f}x_i)^2 - (\sum_{i=1}^{n}v_{i,f}^2 x_i^2)^2)
$$
此时损失函数可以自定义，如回归问题可以MSE，分类问题，可以对数几率。

## 优化

SGD优化（上式凸函数，亦可以导,工程上：FTRL+FM）



## 其他模型优点

线性时间，特征扩展下比svd++，MF等容易。且特征稀疏情况下比svm核方法明显优势。

# FFM

增加field概念，
$$
\varnothing_{FFM} = \sum_{j_i=1}^{n}\sum_{j_2=j_1 + 1}^{n}{w_{j_1,f2}w_{j_2,f1}}x_{j_1}x_{j_2}\\
y = w_0 + \sum_{i=0}^{n}w_ix_i + \varnothing_{FFM}
$$
其中$f_1,f_2$分别表示$j_1,j_2$所属的field。论文例子解释一下：

| Clicked | Publisher(P) | Advertister(A) | Gender(G) |
| ------- | ------------ | -------------- | --------- |
| Yes     | ESPN         | Nike           | Male      |



FM模型角度：$w_{ESPN} \cdot w_{Nike} + w_{WSPN} \cdot w_{Male} + w_{Nike} \cdot w_{Male}$

FFM模型：$w_{ESPN,A} \cdot w_{Nike,P} + w_{WSPN,G} \cdot w_{Male,P} + w_{Nike,G} \cdot w_{Male,A}$

f为fileds数，则FFMS变量数：nfk，上式后半段复杂度为：$O(\bar{n}^2k)$,因为每个潜向量$v$仅需要学习一个特定filed的影响。通常$k_{FFM} << k_{FM}$。

## 优化

使用log loss，只能用于二分类。优化目标：
$$
\underset{w}{min} \frac{\lambda}{2}||w||_2^2 + \sum_{i=1}^{m}log(1 + exp(-y_i \varnothing_{FFM}(\mathbf{w}, \mathbf{x}_i)) ))
$$


AdaGrad,对每个样本，只更新有值特征（普通SGD区别：累计梯度），需要调的参数：学习率

## 总结

隐向量长度k对效果影响不大

相比与FM，速度慢，精度高

参数多，容易过合，所以有正则化，训练也需要早停


# Ref

实对称矩阵性质：https://blog.csdn.net/Banach_I/article/details/51078451

上述证明需要用到：N×N阶实对称矩阵存在N个线性无关向量

AdaGrad参考 https://zhuanlan.zhihu.com/p/29920135 

FM论文：https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf

FFM论文：https://www.csie.ntu.edu.tw/~cjlin/papers/ffm.pdf

LR+FTRL https://zhuanlan.zhihu.com/p/55135954